In [ ]:
%matplotlib notebook
from astropy.table import Table, vstack, join
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from astropy.io.fits import getheader
from utilities import redshifts_from_papers

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

In [ ]:
def load_detcats(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_vtp.detect'):
            data_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            if len(data_part) > 0:
                data_part['Field'] = field
                try:
                    hdr = getheader(f'{outpath}/{field}/{field}_vtp_high.detect', ext=1)
                except FileNotFoundError:
                    continue
                data_part['Field_EXP'] = hdr['EXPOSURE']
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data


def load_MCMCfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/{field}_mcmcfits.txt'):
            try:
                fit_part = Table.read(f'{outpath}/{field}/{field}_mcmcfits.txt', format='ascii', header_start=0)
            except :
                print(field)
                raise (FileNotFoundError)
            src_part = Table.read(f'{outpath}/{field}/{field}_vtp.detect', format='fits')
            
            fit_part.rename_column('ID', 'INDEX')
            try:
                data_part = join(fit_part, src_part, keys='INDEX', join_type='inner')
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

def load_spectralfits(outpath, fields):
    for field in fields:
        if os.path.exists(f'{outpath}/{field}/spec_files/{field}_xspec.results'):
            try:
                data_part = Table.read(f'{outpath}/{field}/spec_files/{field}_xspec.results', 
                                       format='ascii', header_start=0)
                if len(data_part) < 1:
                    continue
                
            except ValueError:
                continue
        else:
            continue

        # handle the first pass when we only have one part
        try:
            data = vstack([data, data_part])
        except NameError:
            data = data_part
        
    return data

In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')
data = redshifts_from_papers(data)

datapath = '../data'
data.replace(-1, np.nan, inplace=True)

fields = [n.replace(' ', '_') for n in data['NAME']]

fits = load_MCMCfits(datapath, fields) # not currently used, but gives access to beta-model params
cats = load_detcats(datapath, fields)
specs = load_spectralfits(datapath, fields)

# check to see which fields have XRT data
field_mask = [True if os.path.isdir(f'{datapath}/{n.replace(" ", "_")}') else False for n in data['NAME']]
                                    
data['SWIFT'] = False
data['SWIFT'] = field_mask

In [ ]:
# add more columns -- don't think I really need these in the end.
# data['XRT_exp'] = np.nan
# data['N_H'] = np.nan

In [ ]:
cats.remove_column('R')
df_cats = cats.to_pandas()
df_cats.Field = df_cats.Field.str.replace('_', ' ')

df_specs = specs.to_pandas()
df_specs.FIELD = df_specs.FIELD.str.replace('_', ' ')

fits.remove_column('R')
df_fits = fits.to_pandas()

In [ ]:
# df_fits

In [ ]:
# df_cats

In [ ]:
# df_specs

In [ ]:
df_data = data[['NAME', 'INDEX_PSZ2', 'RA', 'DEC', 'REDSHIFT']]

In [ ]:
# df_data

In [ ]:
# merge the catalog information to the main "data" frame
df_m = df_data.merge(df_cats,                                                      
               how='inner',                                              
               left_on='NAME',                                          
               right_on='Field',                                           
               suffixes=('_data', '_detc'))

# merge the spectral information to this new merged frame
df_m2 = df_m.merge(df_specs,                                                      
               how='left',                                              
               left_on=['NAME', 'INDEX'],                                          
               right_on=['FIELD', 'INDEX'],                                           
               suffixes=('_data', '_detc'))

In [ ]:
# df_m

In [ ]:
# df_m2

In [ ]:
# only select extended objects that are within 6' of the PSZ position
df_close_ext = df_m2[(df_m2.PSZ_dist < 6) & (df_m2.Extended == 1)]

In [ ]:
df_close_ext.columns

In [ ]:
cols = ['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME',
        'SN', 'redCHI2', 'DOF', 'NORM', 'NORM_NEG', 'NORM_POS', 'FLUX', 'CNT_RATE_NET', 
        'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_CORR', 'LUM_CORR', 'CORR']

In [ ]:
df_close_ext[df_close_ext.FLUX_CORR.isnull()][cols]

In [ ]:
# calculate the errors for the flux and Luminosity
for c in ['FLUX_CORR_POSERR', 'FLUX_CORR_NEGERR', 'LUM_CORR_POSERR', 'LUM_CORR_NEGERR']:
    df_close_ext.loc[:, c] = np.nan

df_close_ext['FLUX_CORR_POSERR'] = df_close_ext['FLUX_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_POS']
df_close_ext['FLUX_CORR_NEGERR'] = df_close_ext['FLUX_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_NEG']
df_close_ext['LUM_CORR_POSERR'] = df_close_ext['LUM_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_POS']
df_close_ext['LUM_CORR_NEGERR'] = df_close_ext['LUM_CORR'] / df_close_ext['NORM'] * df_close_ext['NORM_NEG']

In [ ]:
df_close_ext

In [ ]:
cols = ['NAME', 'RA_detc', 'DEC_detc', 'REDSHIFT', 'INDEX', 'PSZ_dist', 'EXPTIME',
        'SN', 'redCHI2', 'DOF', 'NORM', 'NORM_NEG', 'NORM_POS', 'FLUX', 'CNT_RATE_NET', 
        'CNT_RATE_MODEL', 'ECF', 'LUM', 'FLUX_CORR', 'FLUX_CORR_POSERR', 'FLUX_CORR_NEGERR', 
        'LUM_CORR', 'LUM_CORR_POSERR', 'LUM_CORR_NEGERR', 'CORR']

t1 = Table.from_pandas(df_close_ext[cols])

In [ ]:
t1.filled(-1.0)

In [ ]:
t1.filled(-1.0).write('./catalogs/results_table_full_0.5_2.0.fits', overwrite=True)